In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision.transforms import transforms
from torchsummary import summary

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

超参数

In [ ]:
num_epoch=80
batch_size={'train':2,
            'val':1
            }
#train_path='/content/drive/MyDrive/Data/data/train'
#val_path='/content/drive/MyDrive/Data/data/test'

In [ ]:
from dataset.Fetus import FetusDataset
from model.unet import Unet
from tools.metrics import dice_coef,iou_score,get_accuracy,get_precision,get_specificity,get_recall,get_F1
from tools.utils import AverageMeter

In [ ]:
x_transforms = transforms.Compose([
    transforms.Resize((512,512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
# mask只需要转换为tensor
y_transforms = transforms.Compose([
    transforms.Resize((512, 512)),
    # transforms.CenterCrop(512),
    transforms.ToTensor()
   ])

In [ ]:
train_dataset=FetusDataset(train_path,transform=x_transforms,target_transform=y_transforms)
val_dataset=FetusDataset(val_path,transform=x_transforms,target_transform=y_transforms)
dataloaders={
    'train':DataLoader(train_dataset,batch_size=batch_size['train'],shuffle=True,num_workers=0),
    'val':DataLoader(val_dataset,batch_size=batch_size['val'],shuffle=True,num_workers=0)
             }

In [ ]:
def train_model(model,criterion,optimizer,num_epochs):

    for epoch in range(num_epochs):
        for phase in ['train','val']:

            if phase =='train':
                model.train()
                step=0
                epoch_loss=0
            else:
                model.eval()
            avgmeter1 = AverageMeter()
            avgmeter2 = AverageMeter()
            for idx,data in enumerate(dataloaders[phase]):
                inputs,labels=data[0].to(device),data[1].to(device)
                outputs=model(inputs)
                loss=criterion(outputs,labels)
                with torch.set_grad_enabled(phase=='train'):
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                iou=iou_score(outputs,labels)
                dice=dice_coef(outputs,labels)

                avgmeter1.update(iou, batch_size[phase])
                avgmeter2.update(dice, batch_size[phase])
                if phase=='train':
                    step+=1
                    epoch_loss+=loss.item()
                    print(phase,"\t epoch: %d  ,idx: %d   ,loss: %0.3f   ,miou:   %.3f,maxiou: %.3f    ,miniou: %.3f    ,mdice: %.3f   ,maxdice: %.3f   ,mindice: %.3f   " %(epoch,idx,epoch_loss / step, avgmeter1.avg, avgmeter1.max, avgmeter1.min, avgmeter2.avg, avgmeter2.max,avgmeter2.min)+'\n')
                else:
                    print(phase,"\t epoch: %d  ,idx: %d   ,miou:   %.3f,maxiou: %.3f    ,miniou: %.3f    ,mdice: %.3f   ,maxdice: %.3f   ,mindice: %.3f   " %(epoch,idx, avgmeter1.avg, avgmeter1.max, avgmeter1.min, avgmeter2.avg, avgmeter2.max,avgmeter2.min)+'\n')

In [ ]:
model=Unet(n_class=1)
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
summary(model, input_size=(3, 224, 224))

In [ ]:
train_model(model,criterion,optimizer,num_epoch)
